In [78]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import logging

import time

In [13]:
def parse_message(content):
    """Take the content of a message page, and return the title and body"""
    soup = BeautifulSoup(content, 'html.parser')
    
    message_title = soup.find_all("td",{"class":"Boards-MessageTitle"})[1].text
    message_body = soup.find_all("td",{"class":"Boards-Message"})[0].text
    
    return message_title, message_body


def pull_message(index):
    """"Query a given message ID, and return the content if it exists"""
    url = f"https://secure.marathonguide.com/Forums/CherryBlossomTenMile.cfm?step=4&MID={index}&Topic=175" # noqa

    r = requests.get(url)

    if "invalid URL" in r.text:
        # message doesn't exist
        return
    else:
        return r.content

In [80]:
starting_index = 60875 - 50

In [81]:
import numpy as np

In [82]:
time.sleep(np.random.randint(1,10))

In [83]:
def search_messages(index):
    df = pd.DataFrame()

    next_message = pull_message(index)

    while next_message:
        message_title, message_body = parse_message(next_message)

        logging.info(f'New message: {message_title}\n{message_body}')

        df = df.append(pd.DataFrame([[index, message_title, message_body]],columns=['index', 'title','body']))

        # store latest index
        index += 1

        time.sleep(np.random.randint(1,10))
        # Look for the next (next) message
        next_message = pull_message(index)
    else:
        logging.info("No more messages, storing index and quitting.")
    
    return df

In [84]:
df = search_messages(starting_index)

<ipython-input-83-b00f8a7a7298>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([[index, message_title, message_body]],columns=['index', 'title','body']))
<ipython-input-83-b00f8a7a7298>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([[index, message_title, message_body]],columns=['index', 'title','body']))
<ipython-input-83-b00f8a7a7298>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([[index, message_title, message_body]],columns=['index', 'title','body']))
<ipython-input-83-b00f8a7a7298>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df

In [85]:
df

,index,title,body
0,60825,\r\n10M bib available\r\nChandler\r\n2/14/23 a...,\nPlease email if interested! \r\n\tPost a rep...
0,60826,\r\n5K Bib AVAILABLE\r\nJanelle\r\n2/14/23 at ...,"\nI just had a 5K bib transferred to me, but d..."
0,60827,\r\nLooking for 10 miler bib\r\nJovan\r\n2/14/...,"\nHi everyone,\r\nIn search of a 10 miler bib,..."
0,60828,\r\nLooking for 10 Miler Bib!\r\nKevin Stoll\r...,\nLooking for any 10 miler bib for my fiancee....
0,60829,\r\n10 mile bib Needed- coming in from Louisia...,\nI appreciate any one offering their bib to m...
...,...,...,...
0,60902,\r\n10 miler BIB neded\r\nbabitha\r\n2/16/23 a...,"\nI am looking for a 10 miler BIB, if anyone r..."
0,60903,\r\nSearching for a 10 mile bib - thank you!\r...,\nWould so appreciate a 10 mile bib - thank yo...
0,60904,\r\n5k bib wanted - have 10 miler\r\nlawand\r\...,\nnot feeling up for the 10 miler but want to ...
0,60905,\r\nSeeking 5k bib with medal if available!\r\...,"\nH!\n\r\nSeeking 5k bib. If medal also, that..."


In [86]:
from cleantext import clean

In [87]:
df['body_clean'] = df['body'].apply(lambda x: clean(x.split('\r\n\tPost a reply on the Bulletin Board.')[0]))
df['title_clean'] = df['title'].apply(lambda x: clean(x.split('\n')[1]))

In [88]:
df['title_clean'].values

array(['10m bib available', '5k bib available',
       'looking for 10 miler bib', 'looking for 10 miler bib!',
       '10 mile bib needed- coming in from louisiana',
       'looking for a 10 miler bib',
       '10 miler bib + medal package available',
       'looking for 10 miler entry', 'looking for 10 miler entry',
       'looking for 10 mile bib', 'looking for two 10 mi bibs!',
       'request for 3 10 mile bids',
       'make me run 10 miles - iso 10 mile bib', '10 mile bib available',
       '10k bib available', 'one 10 miler (not 10k!!!!) bib available',
       '2 10 miler bibs available', 'looking for 5k bib!',
       '1 10 mile bible available', '1 5k bib',
       'interested in a 10 mile bib!', 'have: 10-mile bib; need: 5k bib',
       '10 miler bib available for anyone interested',
       '10 mile bib available', 'still have 1 5k bib available!',
       '5k available with medal!', 'interested in 2 10miler bib!',
       'looking for 10 mile bib!', '5k bib available',
       '

In [91]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")


RuntimeError: At least one of TensorFlow 2.0 or PyTorch should be installed. To install TensorFlow 2.0, read the instructions at https://www.tensorflow.org/install/ To install PyTorch, read the instructions at https://pytorch.org/.

In [92]:
df.to_csv('training_data.csv')